This is your private sketch-notebook. Do whatever you want with it. When code is ready to be developed, please just copy and paste it to the final notebook. 

In [2]:
# Plot ad hoc CIFAR10 instances
from keras.datasets import cifar10
from matplotlib import pyplot
from scipy.misc import toimage
# Simple CNN model for CIFAR-10
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline  
from PIL import Image
import glob
import numpy as np
import pandas as pd
from IPython.display import Image
from IPython.display import Image
from IPython.display import Image
from IPython.core.display import HTML 


Using TensorFlow backend.


In [3]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [123]:
# the folder path to the folder with all the fishies in there
folders_path = '../Data/Kaggle/train/*'

# creating a dataframe with the columns below
image_dict = {}

# going through all folders and saving file names in dataframe so that order is always correct
for foldername in glob.glob(folders_path):
    for image_name in glob.glob(foldername + '/*.jpg'):
        # getting image name and fish type and putting it in a dictionary
        fish_type = foldername[-3:]
        image_dict[image_name] = fish_type

image_df = pd.DataFrame(image_dict.items(), columns = ['image_name', 'fish_type']).sample(frac=1) # random sampling
dummy_df = pd.get_dummies(image_df, columns=['fish_type']) # dummify data

dummy_df.head()

,image_name,fish_type_ALB,fish_type_ARK,fish_type_BET,fish_type_DOL,fish_type_HER,fish_type_LAG,fish_type_NoF,fish_type_YFT
3354,../Data/Kaggle/train/DOL/img_02936.jpg,0,0,0,1,0,0,0,0
2905,../Data/Kaggle/train/YFT/img_02550.jpg,0,0,0,0,0,0,0,1
717,../Data/Kaggle/train/BET/img_05974.jpg,0,0,1,0,0,0,0,0
1230,../Data/Kaggle/train/YFT/img_03253.jpg,0,0,0,0,0,0,0,1
1837,../Data/Kaggle/train/ALB/img_06512.jpg,1,0,0,0,0,0,0,0


In [124]:
# Smaller dataset for testing purposes!
dummy_df = dummy_df.iloc[0:101,:]

In [125]:
Image(url= "https://kaggle2.blob.core.windows.net/competitions/kaggle/5568/media/species-ref-key.jpg")

In [126]:
# Count of the image types
image_df.groupby('fish_type').count().sort_values('image_name')

,image_name
fish_type,
LAG,67
DOL,117
ARK,176
BET,200
HER,299
NoF,465
YFT,734
ALB,1719


In [127]:
count = 0 # count, to test the algorithm with 100 pictures
images = []

for filename in dummy_df.image_name:
    img = cv2.imread(filename) # Read file    
    img = cv2.resize(img, (224, 224)) # Resize image to 224 x 224 pixels (dont feel obliged to use this)
    img = img.astype(np.float32) # Change to 32 bit floats (good for neural networks, other floats not as much making sense)

    # "Normalize" images
    for rgb_value in [0,1,2]:
        img[:, :, rgb_value] = img[:, :, rgb_value] / 255.0 # 255 is the RGB maximum
        img[:, :, rgb_value] -= numpy.mean(img[:, :, rgb_value])

    img = img.transpose((2,0,1))

    images.append(img)

# X and y value and labels
x = np.asarray(images)
y = dummy_df.iloc[:,1:9].as_matrix()
labels = dummy_df.image_name.apply(lambda path: path[-13:])

# table, just to be sure that everything goes well
dummy_df['image_bytes'] = images
dummy_df.head()

,image_name,fish_type_ALB,fish_type_ARK,fish_type_BET,fish_type_DOL,fish_type_HER,fish_type_LAG,fish_type_NoF,fish_type_YFT,image_bytes
3354,../Data/Kaggle/train/DOL/img_02936.jpg,0,0,0,1,0,0,0,0,"[[[0.226613, 0.0187695, 0.0932793, 0.112887, 0..."
2905,../Data/Kaggle/train/YFT/img_02550.jpg,0,0,0,0,0,0,0,1,"[[[-0.0122658, -0.00834423, -0.00834423, 0.062..."
717,../Data/Kaggle/train/BET/img_05974.jpg,0,0,1,0,0,0,0,0,"[[[0.56314, 0.551375, 0.555297, 0.56314, 0.539..."
1230,../Data/Kaggle/train/YFT/img_03253.jpg,0,0,0,0,0,0,0,1,"[[[0.707718, 0.695954, 0.699875, 0.707718, 0.6..."
1837,../Data/Kaggle/train/ALB/img_06512.jpg,1,0,0,0,0,0,0,0,"[[[-0.130154, -0.122311, -0.114468, -0.114468,..."


In [133]:
split_no = 90

x_train = x[:split_no]
y_train = y[:split_no]
labels_train = labels[:split_no]

x_test = x[split_no:]
y_test = y[split_no:]
labels_test = labels[split_no:]

In [130]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, 224, 224), border_mode='same', activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same', W_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
# Compile model
epochs = 1 # dont leave it like this, only for testing!
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_9 (Convolution2D)  (None, 32, 224, 224)  896         convolution2d_input_5[0][0]      
____________________________________________________________________________________________________
dropout_9 (Dropout)              (None, 32, 224, 224)  0           convolution2d_9[0][0]            
____________________________________________________________________________________________________
convolution2d_10 (Convolution2D) (None, 32, 224, 224)  9248        dropout_9[0][0]                  
____________________________________________________________________________________________________
maxpooling2d_5 (MaxPooling2D)    (None, 32, 112, 112)  0           convolution2d_10[0][0]           
___________________________________________________________________________________________

In [131]:
# Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), nb_epoch=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
classes = model.predict_proba(x_test, batch_size=32)

# get probability table
proba_df = pd.DataFrame(classes)
proba_df.columns = image_df.groupby('fish_type').count().index # workaround to get column names quickly and in right order

print("Accuracy: %.2f%%" % (scores[1]*100))
print ' '
proba_df

Train on 90 samples, validate on 11 samples
Epoch 1/1
11/11 [==============================] - 0s
Accuracy: 54.55%
 


fish_type,ALB,ARK,BET,DOL,HER,LAG,NoF,YFT
0,0.506756,0.024877,0.058784,0.065291,0.051306,0.038819,0.109125,0.145042
1,0.428844,0.023427,0.058552,0.071125,0.041804,0.033080,0.154019,0.189149
2,0.437535,0.022948,0.063415,0.064843,0.045019,0.035351,0.128865,0.202024
3,0.609962,0.009597,0.037177,0.039952,0.025592,0.016731,0.098356,0.162632
4,0.488781,0.027245,0.060165,0.067221,0.048789,0.037209,0.107406,0.163185
5,0.448581,0.025906,0.067605,0.069368,0.052836,0.039578,0.117511,0.178615
6,0.397859,0.036983,0.075243,0.079164,0.065603,0.052575,0.127660,0.164914
7,0.408537,0.033430,0.072096,0.076143,0.060677,0.048232,0.127109,0.173777
8,0.389521,0.016347,0.062469,0.062859,0.036389,0.025675,0.118288,0.288453
9,0.344012,0.035175,0.081605,0.091208,0.060461,0.049630,0.136820,0.201089


In [142]:
proba_df['img_names'] = labels_test.values

fish_type,ALB,ARK,BET,DOL,HER,LAG,NoF,YFT,img_names
0,0.506756,0.024877,0.058784,0.065291,0.051306,0.038819,0.109125,0.145042,img_05281.jpg
1,0.428844,0.023427,0.058552,0.071125,0.041804,0.033080,0.154019,0.189149,img_01118.jpg
2,0.437535,0.022948,0.063415,0.064843,0.045019,0.035351,0.128865,0.202024,img_07869.jpg
3,0.609962,0.009597,0.037177,0.039952,0.025592,0.016731,0.098356,0.162632,img_01287.jpg
4,0.488781,0.027245,0.060165,0.067221,0.048789,0.037209,0.107406,0.163185,img_01144.jpg
5,0.448581,0.025906,0.067605,0.069368,0.052836,0.039578,0.117511,0.178615,img_02519.jpg
6,0.397859,0.036983,0.075243,0.079164,0.065603,0.052575,0.127660,0.164914,img_05043.jpg
7,0.408537,0.033430,0.072096,0.076143,0.060677,0.048232,0.127109,0.173777,img_04851.jpg
8,0.389521,0.016347,0.062469,0.062859,0.036389,0.025675,0.118288,0.288453,img_05282.jpg
9,0.344012,0.035175,0.081605,0.091208,0.060461,0.049630,0.136820,0.201089,img_01156.jpg


In [ ]:
#fish_subimages = []

#for i in output:
#    if (output[1] in subsetFish & output[2] > .8):
#        fish_subimaged.append(subimage[1])
    
    